# Imports

In [1]:
import pandas as pd
import sklearn.metrics as metrics
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, date, timedelta
import math
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
from time import sleep
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Reshape, TimeDistributed, LSTM, Lambda, Bidirectional, RepeatVector, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import Callback, EarlyStopping, ProgbarLogger, History
import tensorflow as tf
import keras.backend as K
from IPython.display import display
from tensorflow.keras.metrics import get as metric_get
import re
import talos
from copy import deepcopy, copy
import time
import util
import functools 
import itertools
import networkx as nx
from typing import List, Iterator, Tuple, Dict, Union
import json

# Set up GPU:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

# Load data
load hospitalisations and trends

In [2]:
# topics considered
list_topics = {
    'Fièvre': '/m/0cjf0',
    'Mal de gorge': '/m/0b76bty',
    #'Dyspnée': '/m/01cdt5',
    #'Agueusie': '/m/05sfr2',
    #'Anosmie': '/m/0m7pl',
    #'Coronavirus': '/m/01cpyy',
    #'Virus': '/m/0g9pc',
    #'Température corporelle humaine': '/g/1213j0cz',
    #'Épidémie': '/m/0hn9s',
    'Symptôme': '/m/01b_06',
    #'Thermomètre': '/m/07mf1',
    #'Grippe espagnole': '/m/01c751',
    #'Paracétamol': '/m/0lbt3',
    #'Respiration': '/m/02gy9_',
    #'Toux': '/m/01b_21'
}

# hospitalisations features given as input
list_hosp_features = [
    'NEW_HOSP',
    'TOT_HOSP',
    #'TOT_HOSP_log',
    #'TOT_HOSP_pct',
]

europe = False  # if True, use european countries. Otherwise, use french regions and belgium

Additional information: what is the target, should some features remain unscaled?

In [3]:
# target, should be one of the hosp features
target = 'TOT_HOSP'

cumsum = False  # if True, the target will be accumulated over each day

# features that should not be scaled
unscaled = [
    #'NEW_HOSP',
    #'TOT_HOSP',
    #'TOT_HOSP_log',
    #'TOT_HOSP_pct',
    #'Fièvre',
    #'Mal de gorge',
    #'Dyspnée',
    #'Agueusie',
    #'Anosmie',
    #'Coronavirus',
    #'Virus',
    #'Température corporelle humaine',
    #'Épidémie',
    #'Symptôme',
    #'Thermomètre',
    #'Grippe espagnole',
    #'Paracétamol',
    #'Respiration',
    #'Toux',
]

Type of prediction: how many days as input sould be used to predict how many days as output? Should we give a prediction on all days or only on the last?

In [4]:
date_begin = "2020-02-01"
n_forecast = 20
n_samples = 30
predict_one = False

In [5]:
validation_metrics = [metric_get("MeanSquaredError"), metric_get('MeanAbsoluteError'), 
                      metric_get('RootMeanSquaredError')]

url_world = "../data/hospi/world.csv"
url_pop = "../data/population.txt"
url_trends = "../data/trends/model/"
url_hospi_belgium = "../data/hospi/be-covid-hospi.csv"
url_department_france = "france_departements.csv"
url_hospi_france_new = "../data/hospi/fr-covid-hospi.csv"
url_hospi_france_tot = "../data/hospi/fr-covid-hospi-total.csv"
if europe:
    population = util.get_world_population(url_pop)
    renaming = {v: k for k, v in util.european_geocodes.items()}
    geocodes = {k: v for k, v in util.european_geocodes.items() if population[k] > 1_000_000}
    df_hospi = util.hospi_world(url_world, geocodes, renaming, new_hosp=True, date_begin=date_begin)
    augment_population = {k: v/1000 for k, v in population.items()}
else:
    geocodes = util.french_region_and_be
    population = pd.read_csv(url_department_france).groupby('regionTrends').agg({'population': 'sum'})
    augment_population = {k: pop['population'] / 100_000 for k, pop in population.iterrows()}  # pop per 100.000
    df_hospi = util.hospi_french_region_and_be(url_hospi_france_tot, url_hospi_france_new, url_hospi_belgium, 
                                           url_department_france, util.french_region_and_be, new_hosp_in=True, 
                                           tot_hosp=True, date_begin=date_begin)
df_trends = util.create_df_trends(url_trends, list_topics, geocodes)  # TODO deal with augmented data
for k in df_hospi.keys(): # Rolling average of 7 days 
    df_hospi[k] = df_hospi[k].rolling(7, center=True).mean().dropna()
    df_trends[k] = df_trends[k].rolling(7, center=True).mean().dropna()
merged_df = {k: pd.merge(df_hospi[k], df_trends[k], left_index=True, right_index=True).dropna() for k,v in geocodes.items()}
    
scaler_generator = MinMaxScaler
dg = util.DataGenerator(merged_df, n_samples, n_forecast, target, scaler_generator=scaler_generator, scaler_type='batch',
                       augment_merge=2, augment_adjacency=util.france_region_adjacency, augment_population=augment_population,
                       predict_one=predict_one, cumsum=cumsum, data_columns=list_hosp_features)
n_features = dg.n_features
target_idx = dg.target_idx

### Model

In [6]:
if not predict_one:
    weights_loss = np.array([(1/x) for x in range(1, n_forecast+1)])
else:
    weights_loss = 1

def custom_loss_function(y_true, y_pred):
    y_true = y_true * weights_loss
    y_pred = y_pred * weights_loss
    return tf.keras.losses.mean_squared_error(y_true, y_pred)

def get_encoder_decoder(batch_input_shape):
    model = Sequential()
    #model.add(Bidirectional(LSTM(8, return_sequences=True, stateful=False), 
    #                        input_shape=(n_samples, n_features), merge_mode="ave"))
    model.add(LSTM(16, return_sequences=True, stateful=False, batch_input_shape=batch_input_shape, recurrent_dropout=0))
    model.add(LSTM(8, return_sequences=False, stateful=False))
    model.add(RepeatVector(n_forecast))  # repeat
    model.add(LSTM(8, return_sequences=True, stateful=False))  # dec
    if not predict_one:
        model.add(LSTM(16, return_sequences=True, stateful=False))  # dec
        model.add(TimeDistributed(Dense(1)))
        model.add(Reshape((n_forecast,)))
    else:
        model.add(LSTM(16, return_sequences=False, stateful=False))  # dec
        model.add(Dense(1))
        model.add(Reshape((1,)))
    model.compile(loss=custom_loss_function, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model
    
get_encoder_decoder((1, n_samples, n_features)).output_shape

(1, 20)

## Models to beat
### MultiStepLastBaseline
This model repeats the last value of hospitalisations `n_forecast` time

In [7]:
class MultiStepLastBaseline(tf.keras.Model):
    """
    repeat the last hospitalisations given as input n_forecast time
    """
    def call(self, inputs):
        a = inputs[:, -1:, target_idx:target_idx+1]  # target of the last days
        # a = tf.where(tf.not_equal(a, 0), tf.zeros_like(a), a)
        if not predict_one:
            return tf.tile(
                a,
                [1, n_forecast, 1]   # repeat target n_forecast time
            )
        else:
            return tf.tile(a, [1, 1, 1])

def get_baseline():
    model = MultiStepLastBaseline()
    model.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

### Linear regression
linear regression of the last `n_sample` days used to predict the next `n_forecast` days

In [8]:
class LinearRegressionHospi(tf.keras.Model):
    """
    repeat the last hospitalisations given as input n_forecast time
    """
    def predict(self, inputs):
        y = inputs[:, :, target_idx]  # target of the last days
        length = len(inputs)
        x = np.arange(n_samples).reshape(-1,1)  # dates of the target
        if not predict_one:
            result = np.zeros((length, n_forecast))
            for i in range(length):
                regr = LinearRegression().fit(x, y[i])  # linear regression of (days, target)
                result[i] = regr.predict(np.arange(n_samples, n_samples+n_forecast).reshape(-1,1))
        else:
            result = np.zeros((length, 1))
            for i in range(length):
                regr = LinearRegression().fit(x, y[i])
                result[i] = regre.predict([n_samples+n_forecast-1])
        return result
        
def get_custom_linear_regression():
    model = LinearRegressionHospi()
    model.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

### 1 fully connected layer (Dense model)
Using only the target in the fit

In [9]:
DenseModel = Sequential()
DenseModel.add(Lambda(lambda x: x[:,:,target_idx]))  # select only the target of the previous days
if not predict_one:
    DenseModel.add(Dense(n_forecast))   # predict the next target based on the previous ones
else:
    DenseModel.add(Dense(1))
DenseModel.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

def get_dense_model(batch_input_shape):
    model = Sequential()
    model.add(Lambda(lambda x: x[:,:,target_idx], batch_input_shape=batch_input_shape))  # select only the target of the previous days
    model.add(Dense(n_forecast))   # predict the next target based on the previous ones
    model.compile(loss=tf.losses.MeanSquaredError(),
                          metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

# Simple prediction
use a percentage of values for training and the remaining values for testing

In [10]:
# X% for training, remaining for test
ratio_training = 0.9
nb_datapoints = dg.batch_size
max_train = int(ratio_training * nb_datapoints)
train_idx = np.array(range(max_train))
test_idx = np.array(range(max_train, nb_datapoints))
X_train = dg.get_x(train_idx, scaled=True)
Y_train = dg.get_y(train_idx, scaled=True)
X_test_unscaled = dg.get_x(test_idx, scaled=False)
Y_test_unscaled = dg.get_y(test_idx, scaled=False)
X_test = dg.get_x(test_idx, scaled=True, use_previous_scaler=True)
Y_test = dg.get_y(test_idx, scaled=True, use_previous_scaler=True)
Y_test_unpadded_unscaled = dg.remove_padded_y(Y_test_unscaled, idx=test_idx)

### Encoder decoder

In [11]:
batch_input = len(train_idx)
encoder_decoder_train = get_encoder_decoder(batch_input_shape=(batch_input, n_samples, n_features))
encoder_decoder_train.summary()
start = time.time()
encoder_decoder_train.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=batch_input)
end = time.time() - start
print(f"computed in {str(str(timedelta(seconds=end)))} s")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (317, 30, 16)             1216      
_________________________________________________________________
lstm_5 (LSTM)                (317, 8)                  800       
_________________________________________________________________
repeat_vector_1 (RepeatVecto (317, 20, 8)              0         
_________________________________________________________________
lstm_6 (LSTM)                (317, 20, 8)              544       
_________________________________________________________________
lstm_7 (LSTM)                (317, 20, 16)             1600      
_________________________________________________________________
time_distributed_1 (TimeDist (317, 20, 1)              17        
_________________________________________________________________
reshape_1 (Reshape)          (317, 20)                

KeyboardInterrupt: 

In [13]:
batch_input = len(test_idx)
encoder_decoder_prediction = get_encoder_decoder(batch_input_shape=(batch_input, n_samples, n_features) )
encoder_decoder_prediction.set_weights(encoder_decoder_train.get_weights())

Y_predicted = np.squeeze(encoder_decoder_prediction.predict(X_test, batch_size=batch_input))
Y_predicted_unscaled = dg.inverse_transform_y(Y_predicted, idx=test_idx)

print(np.mean(Y_predicted - Y_test, axis=0))
print(np.mean(Y_predicted_unscaled - Y_test_unscaled, axis=0))

Y_predicted_unpadded_unscaled =  dg.remove_padded_y(Y_predicted_unscaled, idx=test_idx)

print(np.mean(abs(Y_predicted_unpadded_unscaled - Y_test_unpadded_unscaled), axis=0))

[0.01182181 0.03292095 0.05239069 0.07486242 0.10219903 0.13145376
 0.15931553 0.18528912 0.21041148 0.23275081 0.25341427 0.27160552
 0.28784532 0.3017836  0.31335848 0.32158675 0.32743756 0.33143487
 0.33252344 0.33246252]
[ 1.8350716   3.14282862  5.36804604  7.84245199 10.50754129 13.16738782
 15.69878695 18.14806111 20.68732077 23.12374419 25.51227921 27.72285965
 29.72751725 31.5347566  33.08889689 34.29457278 35.23735888 35.88072739
 36.19764656 36.38213828]
[ 3.81933408  5.32108304  7.68964482 10.20137742 12.92528528 15.58922774
 18.2787198  20.68907373 23.08679783 25.50786655 27.88145711 30.05661317
 32.12509532 34.01339593 35.6625149  37.14915806 38.34783643 39.41953212
 40.34762178 41.08044124]


### Baseline

In [15]:
baseline_model= get_baseline()
Y_predicted_unscaled = np.squeeze(baseline_model.predict(X_test_unscaled))

print(np.mean(Y_predicted_unscaled - Y_test_unscaled, axis=0))

Y_predicted_unpadded_unscaled =  dg.remove_padded_y(Y_predicted_unscaled, idx=test_idx)

print(np.mean(abs(Y_predicted_unpadded_unscaled - Y_test_unpadded_unscaled), axis=0))

[  0.46074878  -0.15338165  -0.40157006   0.74939613  -2.39673914
  -2.88345411   0.05736714  -0.38828503  -3.97644928  -6.80736716
  -8.93417875 -12.83595009 -16.29206925 -14.67572464 -17.6491546
 -21.76751208 -27.00664252 -30.76871981 -36.3254831  -36.26026571]
[ 97.23976213 121.77371275 121.59357122 122.7349443  128.44000302
 105.06647095  67.53846734 110.9022132  134.43473352 129.85358327
 136.07685939 141.34590485 126.69369166  95.88279133 132.96303825
 152.38068353 148.63060826 152.68435713 156.96830775 143.07008432]


### Dense model

In [18]:
batch_input_shape = (1, n_samples, n_features)
dense_model = get_dense_model(batch_input_shape)
dense_model.summary()
start = time.time()
dense_model.fit(X_train, Y_train, epochs=400, verbose=1)
end = time.time() - start
print(f"computed in {str(str(timedelta(seconds=end)))} s")

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (1, 30)                   0         
_________________________________________________________________
dense_4 (Dense)              (1, 20)                   620       
Total params: 620
Trainable params: 620
Non-trainable params: 0
_________________________________________________________________
Epoch 1/400
233/233 [==============================] - 0s 1ms/step - loss: 1.5908 - mse: 1.5908 - mae: 0.7549 - root_mean_squared_error: 1.2600
Epoch 2/400
233/233 [==============================] - 0s 1ms/step - loss: 1.0971 - mse: 1.0971 - mae: 0.6273 - root_mean_squared_error: 1.0471
Epoch 3/400
233/233 [==============================] - 0s 1ms/step - loss: 0.9083 - mse: 0.9083 - mae: 0.5514 - root_mean_squared_error: 0.9530
Epoch 4/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8290 - ms

233/233 [==============================] - 0s 1ms/step - loss: 0.7915 - mse: 0.7915 - mae: 0.5078 - root_mean_squared_error: 0.8895
Epoch 110/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8169 - mse: 0.8169 - mae: 0.5109 - root_mean_squared_error: 0.9037
Epoch 111/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8016 - mse: 0.8016 - mae: 0.5088 - root_mean_squared_error: 0.8953
Epoch 112/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7935 - mse: 0.7935 - mae: 0.5031 - root_mean_squared_error: 0.8907
Epoch 113/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7977 - mse: 0.7977 - mae: 0.5078 - root_mean_squared_error: 0.8930
Epoch 114/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7903 - mse: 0.7903 - mae: 0.5050 - root_mean_squared_error: 0.8889
Epoch 115/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7931 - mse: 0.7931 - mae: 0.5090 - root_mean_squared_error:

233/233 [==============================] - 0s 1ms/step - loss: 0.8074 - mse: 0.8074 - mae: 0.5105 - root_mean_squared_error: 0.8984
Epoch 222/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7810 - mse: 0.7810 - mae: 0.5009 - root_mean_squared_error: 0.8834
Epoch 223/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8222 - mse: 0.8222 - mae: 0.5133 - root_mean_squared_error: 0.9066
Epoch 224/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7952 - mse: 0.7952 - mae: 0.5062 - root_mean_squared_error: 0.8917
Epoch 225/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8022 - mse: 0.8022 - mae: 0.5108 - root_mean_squared_error: 0.8956
Epoch 226/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8027 - mse: 0.8027 - mae: 0.5081 - root_mean_squared_error: 0.8957
Epoch 227/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7823 - mse: 0.7823 - mae: 0.5039 - root_mean_squared_error:

233/233 [==============================] - 0s 1ms/step - loss: 0.7821 - mse: 0.7821 - mae: 0.5055 - root_mean_squared_error: 0.8843
Epoch 334/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7888 - mse: 0.7888 - mae: 0.5043 - root_mean_squared_error: 0.8880
Epoch 335/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8080 - mse: 0.8080 - mae: 0.5086 - root_mean_squared_error: 0.8988
Epoch 336/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7935 - mse: 0.7935 - mae: 0.5021 - root_mean_squared_error: 0.8905
Epoch 337/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7803 - mse: 0.7803 - mae: 0.4942 - root_mean_squared_error: 0.8828
Epoch 338/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7913 - mse: 0.7913 - mae: 0.5055 - root_mean_squared_error: 0.8895
Epoch 339/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8074 - mse: 0.8074 - mae: 0.5021 - root_mean_squared_error:

In [ ]:
Y_predicted = np.squeeze(dense_model.predict(X_test, batch_size=1))
Y_predicted_unscaled = dg.inverse_transform_y(Y_predicted, idx=test_idx)

print(np.mean(Y_predicted - Y_test, axis=0))
print(np.mean(Y_predicted_unscaled - Y_test_unscaled, axis=0))

Y_predicted_unpadded_unscaled =  dg.remove_padded_y(Y_predicted_unscaled, idx=test_idx)

print(mean_absolute_error(Y_predicted_unpadded_unscaled, Y_test_unpadded_unscaled, multioutput="raw_values"))

In [10]:
def plot_prediction_vs_actual(df_y_real, df_y_predicted, prediction_dates, cur_loc, horizon=1, mode=0):
    """
    plot the prediction done on a specific horizon along with the last points of data before
    """
    if not predict_one:
        if mode == 0:
            horizon_range = range(horizon, horizon+1)
        elif mode == 1:
            horizon_range = range(1, horizon+1)
    else:
        horizon_range = [1]

    df_real = df_y_real[cur_loc]
    df_pred = df_y_predicted[cur_loc]
    prediction_dates = prediction_dates[:, horizon]
    
    for horizon in horizon_range:
        fig = plt.figure(figsize=(6,3))
        expected = df_real[f"{target}(t+{horizon})"].values
        plt.plot(prediction_dates, expected, marker=".", label="True value")
        prediction = df_pred[f"{target}(t+{horizon})"].values

        if not predict_one:
            plot_label = f"Prediction horizon {horizon}"
        else:
            plot_label = f"Prediction horizon {n_forecast}"
        plt.plot(prediction_dates, prediction, marker='.', label=plot_label)

        ax = fig.axes[0]
        # set locator
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
        # set formatter
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
        # set font and rotation for date tick labels
        plt.gcf().autofmt_xdate()
        plt.title(f"Plot true and predicted values for {cur_loc}")
        plt.legend()
        plt.grid()
        plt.show()

In [11]:
class ValidationLogger(ProgbarLogger):
    """
    compute metrics on the validation set, using a different batch size than the training set
    at the end of each epoch, the weights of the training model are used to set the validation model
    the error is computed based on the unscaled and unpadded values
    """
    
    def __init__(self, validation_model, val_batch_size, X_val, Y_val, val_idx, val_geo):
        """
        :param validation_model: validation model ready to be used
        :param X_val: X validation set
        :param Y_val: Y validation set, already unscaled and unpadded
        :param val_idx: validation idx
        :param val_geo: validation geoloc
        """
        super(ValidationLogger, self).__init__(count_mode='steps')
        self.validation_model = validation_model
        self.val_batch_size = val_batch_size
        self.X_val = X_val
        self.Y_val = Y_val
        self.val_idx = val_idx
        self.val_geo = val_geo
    
    def on_epoch_end(self, epoch, logs=None):
        """
        at the end of each epoch, compute the metrics asked on the validation set
        """        
        self.validation_model.set_weights(self.model.get_weights())
        Y_pred = self.validation_model.predict(self.X_val, batch_size=self.val_batch_size)
        Y_pred = dg.inverse_transform_y(Y_pred, geo=self.val_geo, idx=self.val_idx)
        Y_pred = dg.remove_padded_y(Y_pred, geo=self.val_geo, idx=self.val_idx)
        for metric in validation_metrics:
            metric.reset_states()
            metric.update_state(Y_pred, self.Y_val)
            logs[f"val_{metric.name}"] = metric.result().numpy()  # prepend name for validation set
        super(ValidationLogger, self).on_epoch_end(epoch, logs)
        


In [12]:
def compute_error(Y_expected, Y_actual) -> pd.DataFrame:
    MAE = mean_absolute_error(Y_expected, Y_actual, multioutput="raw_values")
    MSE = mean_squared_error(Y_expected, Y_actual, multioutput="raw_values")
    values = {}
    for t in range(n_forecast):
        values[f'MAE(t+{t+1})'] = [MAE[t]]
        values[f'MSE(t+{t+1})'] = [MSE[t]]
    return pd.DataFrame(values)


def walk_forward_evaluation(model_generator: callable, nb_fit_first: int, nb_validation: int, nb_test: int, 
                            epochs: int = 400, plot=False, verbose: int = 1,
                            return_history: bool = False, batch_size_fun: callable = None,
                            es_stop_val: bool = False
                           ) -> Union[pd.DataFrame, List[History]]:
    """
    evaluates a model using a walk forward evaluation: multiples fit are done, each followed by at most nb_test
    to evaluate the model
    :param model_generator: function returning the model to evaluate
    :param nb_fit_first: number of datapoints used for the first fit
    :param nb_validation: number of datapoints used for each evaluation set
    :param nb_test: number of datapoints used for the test set (at most)
    :param epochs: number of epochs used on each fit
    :param callbacks: list of callbacks to use
    :param verbose: verbose level. Passed to fit and used to display the error dataframe
    :param return_history: if True, returns the list of history of each walk
    :param batch_size_fun: function used to compute the batch size based on the X_train tensor
        if not specified, default to batch_size = len(train_idx)
    """
    # initial index used
    max_len = dg.batch_size
    train_idx = np.arange(nb_fit_first)
    valid_idx = np.arange(nb_fit_first, nb_fit_first + nb_validation)
    if nb_fit_first + nb_validation >= max_len:
        finished = True  # no test can be done
    else:
        finished = False  # a test set can be created
        test_idx = np.arange(nb_fit_first + nb_validation, min(nb_fit_first + nb_validation + nb_test, max_len))
    df_error = pd.DataFrame()
    walk = 0
    last_iter = False  # True when the last iteration is reached
    all_history = []
    while not finished:
        X_train = dg.get_x(train_idx)
        Y_train = dg.get_y(train_idx)
        if batch_size_fun is None:
            batch_size = len(train_idx)
        else:
            batch_size = batch_size_fun(X_train)
            
        if len(valid_idx) > 0:
            X_val = dg.get_x(valid_idx, geo=dg.loc_init)
            Y_val_real = dg.get_y(valid_idx, geo=dg.loc_init, scaled=False)
            Y_val_real = dg.remove_padded_y(Y_val_real, geo=dg.loc_init, idx=valid_idx)
            batch_size_val = len(X_val)
            model_validation = model_generator(batch_input_shape=(batch_size_val, n_samples, n_features))
            val_log = ValidationLogger(model_validation, len(X_val), X_val, Y_val_real, valid_idx, dg.loc_init)
            if es_stop_val:
                callbacks = [val_log, EarlyStopping(monitor="val_root_mean_squared_error", patience=25)]
            else:
                callbacks = [val_log]
        else:
            callbacks = None
            
        model_train = model_generator(batch_input_shape=(batch_size, n_samples, n_features))

        history = model_train.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, callbacks=callbacks, verbose=verbose)
        
        # test only on the unaugmented data without padding
        batch_size = len(test_idx)
        # construct a new model and use the previous weights to set it
        model_pred = model_generator(batch_input_shape=(batch_size, n_samples, n_features))
        model_pred.set_weights(model_train.get_weights())
        X_test = dg.get_x(test_idx, geo=dg.loc_init, use_previous_scaler=True)
        Y_test = dg.get_y(test_idx, geo=dg.loc_init, use_previous_scaler=True)
        Y_test_real = dg.get_y(test_idx, geo=dg.loc_init, scaled=False)
        Y_test_real = dg.remove_padded_y(Y_test_real, geo=dg.loc_init, idx=test_idx)
        Y_predicted = model_pred.predict(X_test, batch_size=batch_size)
        Y_predicted_real = dg.inverse_transform_y(Y_predicted, geo=dg.loc_init, idx=test_idx)
        Y_predicted_real = dg.remove_padded_y(Y_predicted_real, geo=dg.loc_init, idx=test_idx)        
        if return_history:
            # add test metrics to the history for this walk, based on the unpadded unscaled data 
            # except for the loss
            for metric in model_train.metrics:
                metric.reset_states()
                if metric.name == 'loss':
                    metric.update_state(Y_predicted_real, Y_test_real)
                else:
                    # compute metric accross each horizon
                    for i in range(n_forecast):
                        metric.update_state(Y_predicted_real[:, i], Y_test_real[:, i])
                        history.history[f"test_{metric.name}(t+{i+1})"] = [metric.result().numpy()]
                        metric.reset_states()
                    # compute mean of metric on all horizon
                    metric.update_state(Y_predicted_real, Y_test_real)
                # prepend name for test set
                history.history[f"test_{metric.name}"] = [metric.result().numpy()]
            # add number of unpadded datapoints
            history.history['nb_test_datapoints'] = [len(Y_test_real)]
            all_history.append(history)
        if not return_history or verbose != 0:
            # compute the error using the unpadded and unscaled data
            error = compute_error(Y_test_real, Y_predicted_real)
            name = f'walk {walk + 1}'
            error['walk'] = name
            error['nb_test_datapoints'] = len(Y_test_real)
            error['days_train'] = len(train_idx)
            error['days_valid'] = len(valid_idx)
            error['days_test'] = len(test_idx)
            error = error.set_index('walk')
            df_error = df_error.append(error)
        if verbose != 0:
            display(df_error)
        if last_iter:
            finished = True
        # indexes for next fit
        train_idx = np.arange(train_idx[-1] + 1 + nb_test)
        valid_idx += nb_test
        if test_idx[-1] + nb_test >= max_len:  # last iteration, less points can be used for the test set
            last_iter = True  # last iteration to be done
            test_idx = np.arange(test_idx[-1], max_len)
        else:
            test_idx += nb_test
        walk += 1
    return df_error if not return_history else all_history

In [15]:
no_div = lambda x : len(x)
df_errors = walk_forward_evaluation(get_dense_model, 250, 10, 30, epochs=300, verbose=1, batch_size_fun=no_div,
                                   es_stop_val=False)

Epoch 1/300
1/1 [==============================] - 0s 177ms/step - loss: 1.6259 - mse: 1.6259 - mae: 0.9620 - root_mean_squared_error: 1.2751 - val_mean_squared_error: 2473278.5000 - val_mean_absolute_error: 924.1101 - val_root_mean_squared_error: 1572.6660
Epoch 2/300
1/1 [==============================] - 0s 22ms/step - loss: 1.5144 - mse: 1.5144 - mae: 0.9266 - root_mean_squared_error: 1.2306 - val_mean_squared_error: 2414139.7500 - val_mean_absolute_error: 913.9866 - val_root_mean_squared_error: 1553.7502
Epoch 3/300
1/1 [==============================] - 0s 21ms/step - loss: 1.4413 - mse: 1.4413 - mae: 0.9033 - root_mean_squared_error: 1.2005 - val_mean_squared_error: 2364063.7500 - val_mean_absolute_error: 905.1722 - val_root_mean_squared_error: 1537.5513
Epoch 4/300
1/1 [==============================] - 0s 22ms/step - loss: 1.3843 - mse: 1.3843 - mae: 0.8849 - root_mean_squared_error: 1.1766 - val_mean_squared_error: 2318699.0000 - val_mean_absolute_error: 897.0258 - val_root_m

Epoch 33/300
1/1 [==============================] - 0s 21ms/step - loss: 0.7486 - mse: 0.7486 - mae: 0.6564 - root_mean_squared_error: 0.8652 - val_mean_squared_error: 1536426.6250 - val_mean_absolute_error: 737.7775 - val_root_mean_squared_error: 1239.5267
Epoch 34/300
1/1 [==============================] - 0s 20ms/step - loss: 0.7362 - mse: 0.7362 - mae: 0.6512 - root_mean_squared_error: 0.8580 - val_mean_squared_error: 1515781.2500 - val_mean_absolute_error: 732.9811 - val_root_mean_squared_error: 1231.1708
Epoch 35/300
1/1 [==============================] - 0s 20ms/step - loss: 0.7242 - mse: 0.7242 - mae: 0.6461 - root_mean_squared_error: 0.8510 - val_mean_squared_error: 1495688.3750 - val_mean_absolute_error: 728.2217 - val_root_mean_squared_error: 1222.9834
Epoch 36/300
1/1 [==============================] - 0s 21ms/step - loss: 0.7125 - mse: 0.7125 - mae: 0.6410 - root_mean_squared_error: 0.8441 - val_mean_squared_error: 1475597.5000 - val_mean_absolute_error: 723.5109 - val_roo

Epoch 65/300
1/1 [==============================] - 0s 28ms/step - loss: 0.4646 - mse: 0.4646 - mae: 0.5182 - root_mean_squared_error: 0.6816 - val_mean_squared_error: 1024740.5000 - val_mean_absolute_error: 608.9827 - val_root_mean_squared_error: 1012.2947
Epoch 66/300
1/1 [==============================] - 0s 21ms/step - loss: 0.4586 - mse: 0.4586 - mae: 0.5148 - root_mean_squared_error: 0.6772 - val_mean_squared_error: 1013382.0625 - val_mean_absolute_error: 605.9100 - val_root_mean_squared_error: 1006.6688
Epoch 67/300
1/1 [==============================] - 0s 29ms/step - loss: 0.4528 - mse: 0.4528 - mae: 0.5113 - root_mean_squared_error: 0.6729 - val_mean_squared_error: 1002048.0000 - val_mean_absolute_error: 602.8127 - val_root_mean_squared_error: 1001.0235
Epoch 68/300
1/1 [==============================] - 0s 21ms/step - loss: 0.4471 - mse: 0.4471 - mae: 0.5081 - root_mean_squared_error: 0.6687 - val_mean_squared_error: 991130.5625 - val_mean_absolute_error: 599.8276 - val_root

KeyboardInterrupt: 

In [83]:
# X% for training, remaining for test
ratio_training = 0.8
nb_datapoints = dg.batch_size
max_train = int(ratio_training * nb_datapoints)
train_idx = np.array(range(max_train))
val_idx = np.array(range(max_train, nb_datapoints))
X_train = dg.get_x(train_idx, scaled=True)
Y_train = dg.get_y(train_idx, scaled=True)
X_val = dg.get_x(val_idx, scaled=True, use_previous_scaler=True)
Y_val_unscaled = dg.get_y(val_idx, geo=dg.loc_init, scaled=False, use_previous_scaler=True)
Y_val_unpadded_unscaled = dg.remove_padded_y(Y_val_unscaled, idx=val_idx, geo=dg.loc_init)

model_generator = get_dense_model

batch_size_train = len(train_idx)
batch_size_val = len(X_val)
model = model_generator(batch_input_shape=(batch_size_train, n_samples, n_features))
model_validation = model_generator(batch_input_shape=(batch_size_val, n_samples, n_features))
val_log = ValidationLogger(model_validation, len(X_val), X_val, Y_val_unpadded_unscaled, val_idx, dg.loc_init)
callbacks = [val_log, EarlyStopping(monitor='val_root_mean_squared_error', mode='min', verbose=1, patience=1)]
    
start = time.time()
history = model.fit(X_train, Y_train, batch_size=batch_size_train, epochs=200, callbacks=callbacks)
end = time.time() - start
print(f"computed in {str(str(timedelta(seconds=end)))} s")

Epoch 1/200
179/179 [==============================] - 0s 1ms/step - loss: 1.1839 - mse: 1.1839 - mae: 0.7759 - root_mean_squared_error: 1.0793 - val_mean_squared_error: 331.9821 - val_mean_absolute_error: 11.6835 - val_root_mean_squared_error: 18.2204
Epoch 2/200
179/179 [==============================] - 0s 1ms/step - loss: 0.4572 - mse: 0.4572 - mae: 0.4603 - root_mean_squared_error: 0.6760 - val_mean_squared_error: 294.8744 - val_mean_absolute_error: 11.0358 - val_root_mean_squared_error: 17.1719
Epoch 3/200
179/179 [==============================] - 0s 1ms/step - loss: 0.3894 - mse: 0.3894 - mae: 0.4078 - root_mean_squared_error: 0.6240 - val_mean_squared_error: 282.7642 - val_mean_absolute_error: 10.7674 - val_root_mean_squared_error: 16.8156
Epoch 4/200
179/179 [==============================] - 0s 1ms/step - loss: 0.3648 - mse: 0.3648 - mae: 0.3878 - root_mean_squared_error: 0.6040 - val_mean_squared_error: 296.3748 - val_mean_absolute_error: 11.0865 - val_root_mean_squared_err

In [13]:
hyper_parameter = {
    'hidden_1': [4, 8, 16],
    'hidden_2': [0, 4, 8, 16],
    'lr': [0.001, 0.01],
    'activation': ['relu', 'elu'],
    'reg': [lambda x: regularizers.l1(l=x), lambda x: None],
    'regw': [5e-4, 1e-3],
    'optimizer': ['Adam', 'RMSprop'],
    'losses': ['mse', 'mae', custom_loss_function],
    'scaling': [MinMaxScaler, StandardScaler],
    'batch_size_div': [1],
    'epochs': [600]
}

def get_encoder_decoder(batch_input_shape, p):
    model = Sequential()
    model.add(LSTM(p['hidden_1'], return_sequences=(p['hidden_2'] != 0), 
                   batch_input_shape=batch_input_shape, kernel_regularizer=p['reg'](p['regw'])))
    if p['hidden_2'] != 0:
        model.add(LSTM(p['hidden_2'], return_sequences=False, kernel_regularizer=p['reg'](p['regw'])))
    model.add(RepeatVector(n_forecast))  # repeat
    if p['hidden_2'] != 0:
        model.add(LSTM(p['hidden_2'], return_sequences=True, kernel_regularizer=p['reg'](p['regw'])))  # dec
    if not predict_one:
        model.add(LSTM(p['hidden_1'], return_sequences=True, kernel_regularizer=p['reg'](p['regw'])))  # dec
        model.add(TimeDistributed(Dense(1, kernel_regularizer=p['reg'](p['regw']), activation=p['activation'])))
        model.add(Reshape((n_forecast,)))
    else:
        model.add(LSTM(p['hidden_1'], return_sequences=False, kernel_regularizer=p['reg'](p['regw'])))  # dec
        model.add(Dense(1, kernel_regularizer=p['reg'](p['regw']), activation=p['activation']))
        model.add(Reshape((1,)))
    model.compile(loss=p["losses"], optimizer=p["optimizer"], metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    K.set_value(model.optimizer.learning_rate, p['lr'])
    return model
    

def talos_walk(x_train, y_train, x_val, y_val, p):
    """
    walk forward evaluation used by talos to determine the best model to keep
    """
    dg.set_scaler(p['scaling'])
    model_generator = lambda batch_input_shape: get_encoder_decoder(batch_input_shape, p)

    batch_size_fun = lambda x : int(x.shape[0] / p['batch_size_div'])
    history_list = walk_forward_evaluation(model_generator, nb_fit_first=250, nb_validation=0, nb_test=30, 
                                      epochs=p['epochs'], plot=False, verbose=0, 
                                      batch_size_fun=batch_size_fun, return_history=True, es_stop_val=True)
    # compute history to return, based on the list of history
    history = History()
    # used to trick talos to compute the right amount of round_epochs. needs to be the first entry of the dict
    # create an array of len == mean of number of epoch of each history
    history.history['ep'] = np.arange(int(np.mean([len(hist.history['loss']) for hist in history_list]))) + 1
    for log in history_list[0].history.keys():
        history.history[log] = [np.mean([hist.history[log][-1] for hist in history_list])]
    return history, model_generator((1, n_samples, n_features))

scan_object = talos.Scan(
    x=[],
    y=[],
    x_val=[],
    y_val=[],
    params=hyper_parameter,
    model=talos_walk,
    experiment_name='trends1', 
    fraction_limit=0.01
)

100%|██████████| 46/46 [14:52:08<00:00, 1163.65s/it]  


In [22]:
Ca marcheanalyze_object = talos.Analyze(scan_object)
print("MAE", analyze_object.low('mae'))
print("MSE", analyze_object.low('mse'))
print("RMSE", analyze_object.low('root_mean_squared_error'))
print("TEST MAE", analyze_object.low('test_mae'))
print("TEST MSE", analyze_object.low('test_mse'))
print("TEST RMSE", analyze_object.low('test_root_mean_squared_error'))
df = analyze_object.table('test_root_mean_squared_error', ascending=True)
print(df.columns)
columns = [f"test_mae(t+{i})" for i in range(1, n_forecast+1)]
df[['hidden_1', 'hidden_2', 'lr', 'batch_size_div', 'scaling', 'test_mse', 'test_mae'] + columns]

MAE 0.021138007286936045
MSE 0.0021799016976729035
RMSE 0.046687012538313866
TEST MAE 11.598709106445312
TEST MSE 368.9996337890625
TEST RMSE 18.94180679321289
Index(['test_mae(t+16)', 'test_root_mean_squared_error(t+20)',
       'test_mse(t+14)', 'test_mae(t+8)', 'batch_size_div', 'hidden_1',
       'test_root_mean_squared_error(t+3)', 'scaling', 'test_mse(t+8)',
       'test_mae(t+18)', 'test_mae(t+6)', 'test_root_mean_squared_error(t+2)',
       'test_mae(t+3)', 'test_mse(t+9)', 'test_loss',
       'root_mean_squared_error', 'test_mae(t+11)',
       'test_root_mean_squared_error(t+10)', 'regw', 'loss', 'test_mae(t+15)',
       'test_root_mean_squared_error(t+14)', 'duration',
       'test_root_mean_squared_error(t+11)', 'test_mae(t+7)', 'test_mae(t+1)',
       'test_root_mean_squared_error(t+7)', 'optimizer', 'test_mse(t+1)',
       'test_mae(t+17)', 'mae', 'test_mse(t+5)', 'end', 'test_mae(t+9)',
       'test_mae(t+10)', 'test_mae', 'test_root_mean_squared_error(t+9)',
       'test

,hidden_1,hidden_2,lr,batch_size_div,scaling,test_mse,test_mae,test_mae(t+1),test_mae(t+2),test_mae(t+3),...,test_mae(t+11),test_mae(t+12),test_mae(t+13),test_mae(t+14),test_mae(t+15),test_mae(t+16),test_mae(t+17),test_mae(t+18),test_mae(t+19),test_mae(t+20)
29,8,16,0.001,1,<class 'sklearn.preprocessing._data.MinMaxScal...,368.999634,11.613309,3.778100,4.935235,5.915179,...,12.376617,13.007839,13.633907,14.263512,14.920258,15.531412,16.099224,16.683821,17.256342,17.777834
12,16,16,0.010,1,<class 'sklearn.preprocessing._data.StandardSc...,377.847046,12.958789,8.388412,9.880347,10.006425,...,13.358921,13.645789,13.990844,14.349859,14.669656,15.053366,15.370562,15.586323,15.798634,15.899250
38,8,4,0.001,179,<class 'sklearn.preprocessing._data.StandardSc...,406.827850,12.593399,6.765717,7.613912,8.365121,...,13.468274,13.845459,14.167676,14.456120,14.748785,15.063616,15.360583,15.663295,16.004938,16.381424
5,8,0,0.010,1,<class 'sklearn.preprocessing._data.MinMaxScal...,443.305725,12.500948,3.693830,4.283959,5.205354,...,13.635902,14.493777,15.292961,16.010576,16.731924,17.372627,18.027727,18.646788,19.265472,19.899242
30,4,4,0.001,1,<class 'sklearn.preprocessing._data.MinMaxScal...,456.099213,12.087252,4.626646,6.715505,8.265052,...,13.107601,13.613174,14.048625,14.454420,14.844568,15.207379,15.532366,15.824340,16.075926,16.363590
33,8,16,0.010,179,<class 'sklearn.preprocessing._data.StandardSc...,483.986328,12.050247,6.499933,6.763069,7.382070,...,12.414900,13.128962,13.799572,14.418126,15.001484,15.375210,15.975498,16.462032,17.175732,17.527330
15,8,16,0.001,179,<class 'sklearn.preprocessing._data.MinMaxScal...,492.530121,13.382237,4.648648,6.005950,7.197309,...,14.322783,14.980151,15.623283,16.238644,16.897707,17.515383,18.128284,18.703037,19.297628,19.869844
21,4,0,0.010,1,<class 'sklearn.preprocessing._data.MinMaxScal...,492.417572,12.767149,10.886259,10.267223,9.985346,...,13.149528,13.611848,13.995646,14.344925,14.654423,14.907431,15.129011,15.319042,15.537729,15.761189
23,8,4,0.001,179,<class 'sklearn.preprocessing._data.MinMaxScal...,513.338318,13.462949,5.290785,6.213699,7.302274,...,13.636876,14.471559,15.364357,16.282177,17.208750,18.127089,19.030231,19.875698,20.663685,21.355709
36,4,0,0.001,179,<class 'sklearn.preprocessing._data.StandardSc...,555.854492,11.598709,5.281905,5.901314,6.462759,...,11.540359,12.110660,12.728726,13.400656,14.205097,15.057336,15.980198,16.984249,18.057579,19.119024


# 2 estimators combined
The first estimator should be simple and output an estimate $ \hat{y}_{(1)}(t) $ of $ y(t) $
The correction factor ($ C(t) $) between the estimation and the real value is then computed:

$$ y(t) = \hat{y}_{(1)}(t) * ( 1 + C(t)) $$

$$ C(t) = \frac{y(t) - \hat{y}_{(1)}(t)}{\hat{y}_{(1)}(t)} $$

This formula is used such that $C(t)$ should be bounded in [-1, 1] if the target is a positive value and the first estimator is relatively closed to the expected value.

Anoter estimator is then trained based on the first one. The goal is to be able to predict the correction $ C(t) $ that should be done on the first estimator, in order to reduce the estimation error and get a better prediction. The prediction of $ C(t) $ is written $ \hat{C}(t) $. Once this estimator is computed, the final prediction can be constructed:

$$ \hat{y}(t) = \hat{y}_{(1)}(t) * ( 1 + \hat{C}(t)) $$



## First estimator
Construct the predition $\hat{y}_{(1)}(t)$ and compute $C(t)$ based on it

In [13]:
# X% for training, remaining for test
ratio_training = 0.8
epochs = 200

nb_datapoints = dg.batch_size
max_train = int(ratio_training * nb_datapoints)
train_idx = np.array(range(max_train))
valid_idx = np.array([])
test_idx = np.array(range(max_train, nb_datapoints))

X_train_1 = dg.get_x(train_idx, scaled=True)
Y_train = dg.get_y(train_idx, scaled=True)

if len(valid_idx) > 0:
    X_val_1 = dg.get_x(val_idx, geo=dg.loc_init, scaled=True, use_previous_scaler=True)
    Y_val_unscaled = dg.get_y(val_idx, geo=dg.loc_init, scaled=False)
    Y_val_real = dg.remove_padded_y(Y_val_unscaled, idx=val_idx, geo=dg.loc_init)

X_test_1 = dg.get_x(test_idx, scaled=True, geo=dg.loc_init, use_previous_scaler=True)
Y_test_unscaled = dg.get_y(test_idx, scaled=False, geo=dg.loc_init)
Y_test_real = dg.remove_padded_y(Y_test_unscaled, idx=test_idx, geo=dg.loc_init)
    
"""
X_train_val = dg.get_x(train_val_idx, scaled=True)
df_Y_real = {loc: dg.df[loc][dg.target_columns].iloc[train_val_idx] for loc in dg.df}
Y_real_train_val = dg.get_y(train_val_idx, scaled=True, use_previous_scaler=True)
"""

model_generator = get_dense_model

batch_size_train = len(X_train_1)
batch_size_test = len(X_test_1)

if len(valid_idx) > 0:  # use validation set for an early stop
    batch_size_val = len(X_val_1)
    model_validation = model_generator(batch_input_shape=(batch_size_val, n_samples, dg.n_features))
    val_log = ValidationLogger(model_validation, len(X_val), X_val, Y_val_real, valid_idx, dg.loc_init)
    callbacks = [val_log, EarlyStopping(monitor='val_root_mean_squared_error', mode='min', verbose=1, patience=20)]
else:
    callbacks = None

model = model_generator(batch_input_shape=(batch_size_train, n_samples, dg.n_features))
history = model.fit(X_train_1, Y_train, batch_size=batch_size_train, epochs=epochs, callbacks=callbacks)
# compute the predictions on both the training and the validation
Y_train_pred_1 = model.predict(X_train_1, batch_size=batch_size_train)

model_prediction = model_generator(batch_input_shape=(batch_size_test, n_samples, dg.n_features))
model_prediction.set_weights(model.get_weights())
Y_test_pred_1 = model_prediction.predict(X_test_1, batch_size=batch_size_test)
# unscale and unpad the data
Y_test_pred_unscaled_1 = dg.inverse_transform_y(Y_test_pred_1, idx=test_idx, geo=dg.loc_init)
Y_test_pred_real_1 = dg.remove_padded_y(Y_test_pred_unscaled_1, idx=test_idx, geo=dg.loc_init)

"""
df_predicted_1 = dg.inverse_transform_y(Y_train_val_pred, idx=train_val_idx, return_type='dict_df')
df_predicted_1

# predictions on the training set
model_prediction = model_generator(batch_input_shape=(batch_size_train, n_samples, dg.n_features))
model_prediction.set_weights(model.get_weights())
Y_train_pred = model_prediction.predict(X_train, batch_size=batch_size_train)

# predictions on the validation without augmented regions
model_prediction = model_generator(batch_input_shape=(batch_size_val, n_samples, dg.n_features))
model_prediction.set_weights(model.get_weights())
Y_val_pred = model_prediction.predict(X_val, batch_size=batch_size_val)
df_Y_real = {loc: dg.df[loc][dg.target_columns].iloc[train_val_idx] for loc in dg.df}
"""

Epoch 1/200
1/1 [==============================] - 0s 484ms/step - loss: 0.4527 - mse: 0.4527 - mae: 0.4760 - root_mean_squared_error: 0.6728
Epoch 2/200
1/1 [==============================] - 0s 3ms/step - loss: 0.4113 - mse: 0.4113 - mae: 0.4485 - root_mean_squared_error: 0.6413
Epoch 3/200
1/1 [==============================] - 0s 3ms/step - loss: 0.3840 - mse: 0.3840 - mae: 0.4300 - root_mean_squared_error: 0.6197
Epoch 4/200
1/1 [==============================] - 0s 3ms/step - loss: 0.3626 - mse: 0.3626 - mae: 0.4154 - root_mean_squared_error: 0.6022
Epoch 5/200
1/1 [==============================] - 0s 2ms/step - loss: 0.3447 - mse: 0.3447 - mae: 0.4029 - root_mean_squared_error: 0.5871
Epoch 6/200
1/1 [==============================] - 0s 3ms/step - loss: 0.3291 - mse: 0.3291 - mae: 0.3919 - root_mean_squared_error: 0.5736
Epoch 7/200
1/1 [==============================] - 0s 2ms/step - loss: 0.3151 - mse: 0.3151 - mae: 0.3819 - root_mean_squared_error: 0.5613
Epoch 8/200
1/1 [=

1/1 [==============================] - 0s 3ms/step - loss: 0.0659 - mse: 0.0659 - mae: 0.1643 - root_mean_squared_error: 0.2568
Epoch 60/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0645 - mse: 0.0645 - mae: 0.1625 - root_mean_squared_error: 0.2539
Epoch 61/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0630 - mse: 0.0630 - mae: 0.1608 - root_mean_squared_error: 0.2511
Epoch 62/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0617 - mse: 0.0617 - mae: 0.1591 - root_mean_squared_error: 0.2483
Epoch 63/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0603 - mse: 0.0603 - mae: 0.1575 - root_mean_squared_error: 0.2456
Epoch 64/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0591 - mse: 0.0591 - mae: 0.1559 - root_mean_squared_error: 0.2430
Epoch 65/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0578 - mse: 0.0578 - mae: 0.1543 - root_mean_squared_error: 0.2405
Epoch 66/200
1/1 [========

1/1 [==============================] - 0s 2ms/step - loss: 0.0297 - mse: 0.0297 - mae: 0.1122 - root_mean_squared_error: 0.1724
Epoch 118/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0295 - mse: 0.0295 - mae: 0.1116 - root_mean_squared_error: 0.1716
Epoch 119/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0292 - mse: 0.0292 - mae: 0.1112 - root_mean_squared_error: 0.1709
Epoch 120/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0289 - mse: 0.0289 - mae: 0.1107 - root_mean_squared_error: 0.1701
Epoch 121/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0287 - mse: 0.0287 - mae: 0.1102 - root_mean_squared_error: 0.1694
Epoch 122/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0284 - mse: 0.0284 - mae: 0.1097 - root_mean_squared_error: 0.1686
Epoch 123/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0282 - mse: 0.0282 - mae: 0.1093 - root_mean_squared_error: 0.1679
Epoch 124/200
1/1 [=

1/1 [==============================] - 0s 3ms/step - loss: 0.0206 - mse: 0.0206 - mae: 0.0959 - root_mean_squared_error: 0.1434
Epoch 175/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0205 - mse: 0.0205 - mae: 0.0959 - root_mean_squared_error: 0.1431
Epoch 176/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0204 - mse: 0.0204 - mae: 0.0957 - root_mean_squared_error: 0.1428
Epoch 177/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0203 - mse: 0.0203 - mae: 0.0956 - root_mean_squared_error: 0.1425
Epoch 178/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0202 - mse: 0.0202 - mae: 0.0955 - root_mean_squared_error: 0.1423
Epoch 179/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0202 - mse: 0.0202 - mae: 0.0955 - root_mean_squared_error: 0.1420
Epoch 180/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0201 - mse: 0.0201 - mae: 0.0952 - root_mean_squared_error: 0.1418
Epoch 181/200
1/1 [=

"\ndf_predicted_1 = dg.inverse_transform_y(Y_train_val_pred, idx=train_val_idx, return_type='dict_df')\ndf_predicted_1\n\n# predictions on the training set\nmodel_prediction = model_generator(batch_input_shape=(batch_size_train, n_samples, dg.n_features))\nmodel_prediction.set_weights(model.get_weights())\nY_train_pred = model_prediction.predict(X_train, batch_size=batch_size_train)\n\n# predictions on the validation without augmented regions\nmodel_prediction = model_generator(batch_input_shape=(batch_size_val, n_samples, dg.n_features))\nmodel_prediction.set_weights(model.get_weights())\nY_val_pred = model_prediction.predict(X_val, batch_size=batch_size_val)\ndf_Y_real = {loc: dg.df[loc][dg.target_columns].iloc[train_val_idx] for loc in dg.df}\n"

In [14]:
# compute the corrections and add them to a dataframe
data_dg_2 = [f'{topic}(t{i})' for i in range(-n_samples+1, 0, 1) for topic in list_topics] + [topic for topic in list_topics]
target_df_2 = [f'{target}(t+{i})' for i in range(1, n_forecast+1)]
df_c = {loc : dg.df[loc][data_dg_2 + target_df_2] for loc in dg.df}
"""
for k in dg.df:
    # add the data columns to the dataframe
    df_c[k] = dg.df[k][data_dg_2]
    for t in range(1, n_forecast+1):  # add the target columns
        target_t = f'{target}(t+{t})'
        df_c[k][f'C(t+{t})'] = (df_Y_real[k][target_t] - df_predicted_1[k][target_t]) / df_predicted_1[k][target_t]
"""

"\nfor k in dg.df:\n    # add the data columns to the dataframe\n    df_c[k] = dg.df[k][data_dg_2]\n    for t in range(1, n_forecast+1):  # add the target columns\n        target_t = f'{target}(t+{t})'\n        df_c[k][f'C(t+{t})'] = (df_Y_real[k][target_t] - df_predicted_1[k][target_t]) / df_predicted_1[k][target_t]\n"

In [15]:
df_c['BE']

Fièvre(t-29)  Mal de gorge(t-29)  Symptôme(t-29)  \
LOC DATE                                                           
BE  2020-03-04     22.444890           18.401937       19.524793   
    2020-03-05     20.813055           17.578692       19.672373   
    2020-03-06     18.436874           16.416465       18.816411   
    2020-03-07     17.492127           16.900726       18.373672   
    2020-03-08     15.803035           19.305892       17.591499   
...                      ...                 ...             ...   
    2021-02-15     13.199598           18.310267       20.119596   
    2021-02-16     14.563217           17.469796       20.679904   
    2021-02-17     14.186428           17.433776       21.832724   
    2021-02-18     14.892161           19.366859       22.174061   
    2021-02-19     15.849087           19.306826       22.032374   

                Fièvre(t-28)  Mal de gorge(t-28)  Symptôme(t-28)  \
LOC DATE                                                           
BE  2020-03-04     20.813055           17.578692       19.672373   
    2020-03-05     18.436874           16.416465       18.816411   
    2020-03-06     17.492127           16.900726       18.373672   
    2020-03-07     15.803035           19.305892       17.591499   
    2020-03-08     14.614944           17.401130       17.753837   
...                      ...                 ...             ...   
    2021-02-15     14.563217           17.469796       20.679904   
    2021-02-16     14.186428           17.433776       21.832724   
    2021-02-17     14.892161           19.366859       22.174061   
    2021-02-18     15.849087           19.306826       22.032374   
    2021-02-19     15.759375           20.531513       21.581551   

                Fièvre(t-27)  Mal de gorge(t-27)  Symptôme(t-27)  \
LOC DATE                                                           
BE  2020-03-04     18.436874           16.416465       18.816411   
    2020-03-05     17.492127           16.900726       18.373672   
    2020-03-06     15.803035           19.305892       17.591499   
    2020-03-07     14.614944           17.401130       17.753837   
    2020-03-08     14.185514           17.223567       17.163518   
...                      ...                 ...             ...   
    2021-02-15     14.186428           17.433776       21.832724   
    2021-02-16     14.892161           19.366859       22.174061   
    2021-02-17     15.849087           19.306826       22.032374   
    2021-02-18     15.759375           20.531513       21.581551   
    2021-02-19     15.185219           19.582981       21.633073   

                Fièvre(t-26)  ...  TOT_HOSP(t+11)  TOT_HOSP(t+12)  \
LOC DATE                      ...                                   
BE  2020-03-04     17.492127  ...      254.142857      374.714286   
    2020-03-05     15.803035  ...      374.714286      532.000000   
    2020-03-06     14.614944  ...      532.000000      729.714286   
    2020-03-07     14.185514  ...      729.714286      926.857143   
    2020-03-08     14.142571  ...      926.857143     1143.000000   
...                      ...  ...             ...             ...   
    2021-02-15     14.892161  ...     1817.857143     1845.142857   
    2021-02-16     15.849087  ...     1845.142857     1866.714286   
    2021-02-17     15.759375  ...     1866.714286     1887.571429   
    2021-02-18     15.185219  ...     1887.571429     1898.571429   
    2021-02-19     15.209142  ...     1898.571429     1905.000000   

                TOT_HOSP(t+13)  TOT_HOSP(t+14)  TOT_HOSP(t+15)  \
LOC DATE                                                         
BE  2020-03-04      532.000000      729.714286      926.857143   
    2020-03-05      729.714286      926.857143     1143.000000   
    2020-03-06      926.857143     1143.000000     1377.714286   
    2020-03-07     1143.000000     1377.714286     1674.142857   
    2020-03-08     1377.714286     1674.142857     1993.142857   
.

In [16]:
scaler_generator = MinMaxScaler
dg_2 = util.DataGenerator(df_c, n_samples, n_forecast, target=target, scaler_generator=scaler_generator, 
                          scaler_type='batch', augment_merge=0, predict_one=False, cumsum=False,
                          data_columns=[k for k in list_topics], no_lag=True)
dg_2.set_loc_init(dg.loc_init)  # consider the other localisations as being augmented
n_features = dg_2.n_features

## Train the second estimator to compute $Y_2(t)$

In [17]:
X_train_2 = dg_2.get_x(train_idx, scaled=True)
Y_train_2 = dg_2.get_y(train_idx, scaled=True)

if len(valid_idx) > 0:
    X_val_2 = dg_2.get_x(valid_idx, scaled=True, use_previous_scaler=True, geo=dg_2.loc_init)
    Y_val_unscaled_2 = dg.get_y(valid_idx, geo=dg.loc_init, scaled=False)
    Y_val_real_2 = dg.remove_padded_y(Y_val_unscaled, idx=val_idx, geo=dg.loc_init)
    batch_size_val = len(X_val_2)
    
X_test_2 = dg_2.get_x(test_idx, scaled=True, use_previous_scaler=True, geo=dg_2.loc_init)
Y_test_unscaled_2 = dg.get_y(test_idx, scaled=False, geo=dg_2.loc_init)
Y_test_real_2 = dg.remove_padded_y(Y_test_unscaled_2, idx=test_idx, geo=dg_2.loc_init)

model_generator = get_dense_model

batch_size_train = len(X_train_2)
batch_size_test = len(X_test_2)
model = model_generator(batch_input_shape=(batch_size_train, n_samples, dg_2.n_features))
    
history = model.fit(X_train_2, Y_train_2, batch_size=batch_size_train, epochs=epochs, callbacks=None)
# compute the predictions on both the training and the validation
Y_train_pred_2 = model.predict(X_train_2, batch_size=batch_size_train)

model_prediction = model_generator(batch_input_shape=(batch_size_test, n_samples, dg_2.n_features))
model_prediction.set_weights(model.get_weights())
Y_test_pred_2 = model_prediction.predict(X_test_2, batch_size=batch_size_test)
# unscale and unpad the data
Y_test_pred_unscaled_2 = dg_2.inverse_transform_y(Y_test_pred_2, idx=test_idx, geo=dg_2.loc_init)
Y_test_pred_real_2 = dg.remove_padded_y(Y_test_pred_unscaled_2, idx=test_idx, geo=dg_2.loc_init)

Epoch 1/200
1/1 [==============================] - 0s 131ms/step - loss: 0.3553 - mse: 0.3553 - mae: 0.4494 - root_mean_squared_error: 0.5961
Epoch 2/200
1/1 [==============================] - 0s 3ms/step - loss: 0.3276 - mse: 0.3276 - mae: 0.4300 - root_mean_squared_error: 0.5723
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 0.3094 - mse: 0.3094 - mae: 0.4171 - root_mean_squared_error: 0.5562
Epoch 4/200
1/1 [==============================] - 0s 3ms/step - loss: 0.2952 - mse: 0.2952 - mae: 0.4068 - root_mean_squared_error: 0.5433
Epoch 5/200
1/1 [==============================] - 0s 3ms/step - loss: 0.2833 - mse: 0.2833 - mae: 0.3981 - root_mean_squared_error: 0.5323
Epoch 6/200
1/1 [==============================] - 0s 3ms/step - loss: 0.2729 - mse: 0.2729 - mae: 0.3904 - root_mean_squared_error: 0.5224
Epoch 7/200
1/1 [==============================] - 0s 3ms/step - loss: 0.2636 - mse: 0.2636 - mae: 0.3835 - root_mean_squared_error: 0.5134
Epoch 8/200
1/1 [=

1/1 [==============================] - 0s 3ms/step - loss: 0.1004 - mse: 0.1004 - mae: 0.2472 - root_mean_squared_error: 0.3168
Epoch 60/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0994 - mse: 0.0994 - mae: 0.2463 - root_mean_squared_error: 0.3154
Epoch 61/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0985 - mse: 0.0985 - mae: 0.2455 - root_mean_squared_error: 0.3139
Epoch 62/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0977 - mse: 0.0977 - mae: 0.2446 - root_mean_squared_error: 0.3125
Epoch 63/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0968 - mse: 0.0968 - mae: 0.2439 - root_mean_squared_error: 0.3112
Epoch 64/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0960 - mse: 0.0960 - mae: 0.2431 - root_mean_squared_error: 0.3098
Epoch 65/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0952 - mse: 0.0952 - mae: 0.2424 - root_mean_squared_error: 0.3086
Epoch 66/200
1/1 [========

1/1 [==============================] - 0s 3ms/step - loss: 0.0765 - mse: 0.0765 - mae: 0.2235 - root_mean_squared_error: 0.2766
Epoch 118/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0764 - mse: 0.0764 - mae: 0.2232 - root_mean_squared_error: 0.2764
Epoch 119/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0762 - mse: 0.0762 - mae: 0.2232 - root_mean_squared_error: 0.2761
Epoch 120/200
1/1 [==============================] - 0s 4ms/step - loss: 0.0761 - mse: 0.0761 - mae: 0.2229 - root_mean_squared_error: 0.2758
Epoch 121/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0759 - mse: 0.0759 - mae: 0.2227 - root_mean_squared_error: 0.2755
Epoch 122/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0758 - mse: 0.0758 - mae: 0.2225 - root_mean_squared_error: 0.2753
Epoch 123/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0756 - mse: 0.0756 - mae: 0.2223 - root_mean_squared_error: 0.2750
Epoch 124/200
1/1 [=

1/1 [==============================] - 0s 3ms/step - loss: 0.0708 - mse: 0.0708 - mae: 0.2168 - root_mean_squared_error: 0.2661
Epoch 175/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0708 - mse: 0.0708 - mae: 0.2165 - root_mean_squared_error: 0.2660
Epoch 176/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0707 - mse: 0.0707 - mae: 0.2167 - root_mean_squared_error: 0.2659
Epoch 177/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0706 - mse: 0.0706 - mae: 0.2164 - root_mean_squared_error: 0.2658
Epoch 178/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0706 - mse: 0.0706 - mae: 0.2166 - root_mean_squared_error: 0.2657
Epoch 179/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0705 - mse: 0.0705 - mae: 0.2162 - root_mean_squared_error: 0.2656
Epoch 180/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0705 - mse: 0.0705 - mae: 0.2165 - root_mean_squared_error: 0.2655
Epoch 181/200
1/1 [=

## Construct the final prediction by combining the two

In [22]:
class AssembleLayerTimeDist(tf.keras.Model):
    """
    assemble the prediction from the trends and the predictions from another model
    the same weight is used at every timestep (1 trainable parameter)
    """
    def __init__(self, input_shape):
        super(AssembleLayer, self).__init__(name='')
        self.dense_trends = tf.keras.layers.Dense(1, use_bias=False, 
                                                 kernel_constraint=tf.keras.constraints.MinMaxNorm(0.001, 0.2))
        batch_input_shape = (input_shape[0], input_shape[1], 1)
        self.time_dist = TimeDistributed(self.dense_trends, batch_input_shape=batch_input_shape)

    def call(self, input_tensor, training=False):
        x_trends = input_tensor[:, :, 1:]
        x_hosp = input_tensor[:, :, :1]
        x_trends = tf.keras.layers.Subtract()([x_trends, x_hosp])  # x_trends - x_hosp
        x_trends = self.time_dist(x_trends, training=training)  # apply simple weight
        return tf.keras.layers.Add()([x_trends, x_hosp])  # final prediction = x_hosp + (x_trends - x_hosp) * c

class AssembleLayer(tf.keras.Model):
    """
    assemble the prediction from the trends and the predictions from another model
    different weight are used at every timestep (n_forecast trainable parameter)
    """
    
    def __init__(self, batch_input_shape):
        super(AssembleLayer, self).__init__(name='')
        self.kernel = self.add_weight("kernel", shape=[1,batch_input_shape[1]], 
                                      constraint=tf.keras.constraints.MinMaxNorm(0.001, 0.2))

    """
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel",
                              shape=[1,input_shape[1]], constraint=tf.keras.constraints.MinMaxNorm(0.001, 0.2))
    """

    def call(self, input_tensor, training=False):
        x_trends = input_tensor[:, :, 1]
        x_hosp = input_tensor[:, :, 0]
        x_trends = tf.keras.layers.Subtract()([x_trends, x_hosp])  # x_trends - x_hosp
        x_trends = tf.multiply(x_trends, self.kernel)  # apply simple weight: (x_trends - x_hosp) * c
        # x_trends = tf.reshape(x_trends, x_hosp.shape)
        return tf.keras.layers.Add()([x_hosp, x_trends])  # final prediction = x_hosp + (x_trends - x_hosp) * c

    
def get_assemble(batch_input_shape):
    model = AssembleLayer(batch_input_shape)
    model.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

In [23]:
X_train_assembled = np.stack([Y_train_pred_1, Y_train_pred_2], axis=2)
Y_train_assembled = Y_train  # output the target of the first model
X_test_assembled = np.stack([Y_test_pred_1, Y_test_pred_2], axis=2)
Y_test_assembled_real = Y_test_real

batch_size_train = len(X_train_assembled)
batch_size_test = len(X_test_assembled)

model_generator = get_assemble

model_train = model_generator(batch_input_shape=(batch_size_train, n_forecast, 2))
model_train.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
history = model_train.fit(X_train_assembled, Y_train_assembled, batch_size=batch_size_train, epochs=epochs)
Y_train_assembled_pred = model_train.predict(X_train_assembled, batch_size=batch_size_train)

model_prediction = model_generator(batch_input_shape=(batch_size_test, n_forecast, 2))
model_prediction.set_weights(model_train.get_weights())
Y_test_assembled_pred = model_prediction.predict(X_test_assembled, batch_size=batch_size_test)
# unscale and unpad the data
Y_test_assembled_pred_unscaled = dg_2.inverse_transform_y(Y_test_assembled_pred, idx=test_idx, geo=dg.loc_init)
Y_test_assembled_pred_real = dg.remove_padded_y(Y_test_assembled_pred_unscaled, idx=test_idx, geo=dg.loc_init)

Epoch 1/200
1/1 [==============================] - 0s 147ms/step - loss: 0.0228 - mse: 0.0228 - mae: 0.1083 - root_mean_squared_error: 0.1511
Epoch 2/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0199 - mse: 0.0199 - mae: 0.0979 - root_mean_squared_error: 0.1411
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0199 - mse: 0.0199 - mae: 0.0978 - root_mean_squared_error: 0.1410
Epoch 4/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0199 - mse: 0.0199 - mae: 0.0977 - root_mean_squared_error: 0.1409
Epoch 5/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0198 - mse: 0.0198 - mae: 0.0976 - root_mean_squared_error: 0.1408
Epoch 6/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0198 - mse: 0.0198 - mae: 0.0975 - root_mean_squared_error: 0.1408
Epoch 7/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0198 - mse: 0.0198 - mae: 0.0975 - root_mean_squared_error: 0.1407
Epoch 8/200
1/1 [=

1/1 [==============================] - 0s 2ms/step - loss: 0.0192 - mse: 0.0192 - mae: 0.0951 - root_mean_squared_error: 0.1386
Epoch 60/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0192 - mse: 0.0192 - mae: 0.0951 - root_mean_squared_error: 0.1385
Epoch 61/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0192 - mse: 0.0192 - mae: 0.0951 - root_mean_squared_error: 0.1385
Epoch 62/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0192 - mse: 0.0192 - mae: 0.0950 - root_mean_squared_error: 0.1385
Epoch 63/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0192 - mse: 0.0192 - mae: 0.0950 - root_mean_squared_error: 0.1385
Epoch 64/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0192 - mse: 0.0192 - mae: 0.0950 - root_mean_squared_error: 0.1384
Epoch 65/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0192 - mse: 0.0192 - mae: 0.0949 - root_mean_squared_error: 0.1384
Epoch 66/200
1/1 [========

1/1 [==============================] - 0s 2ms/step - loss: 0.0188 - mse: 0.0188 - mae: 0.0936 - root_mean_squared_error: 0.1372
Epoch 118/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0188 - mse: 0.0188 - mae: 0.0936 - root_mean_squared_error: 0.1372
Epoch 119/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0188 - mse: 0.0188 - mae: 0.0936 - root_mean_squared_error: 0.1372
Epoch 120/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0188 - mse: 0.0188 - mae: 0.0936 - root_mean_squared_error: 0.1372
Epoch 121/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0188 - mse: 0.0188 - mae: 0.0936 - root_mean_squared_error: 0.1372
Epoch 122/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0188 - mse: 0.0188 - mae: 0.0936 - root_mean_squared_error: 0.1372
Epoch 123/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0188 - mse: 0.0188 - mae: 0.0935 - root_mean_squared_error: 0.1371
Epoch 124/200
1/1 [=

1/1 [==============================] - 0s 3ms/step - loss: 0.0187 - mse: 0.0187 - mae: 0.0930 - root_mean_squared_error: 0.1366
Epoch 175/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0187 - mse: 0.0187 - mae: 0.0930 - root_mean_squared_error: 0.1366
Epoch 176/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0187 - mse: 0.0187 - mae: 0.0930 - root_mean_squared_error: 0.1366
Epoch 177/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0187 - mse: 0.0187 - mae: 0.0930 - root_mean_squared_error: 0.1366
Epoch 178/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0187 - mse: 0.0187 - mae: 0.0930 - root_mean_squared_error: 0.1366
Epoch 179/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0187 - mse: 0.0187 - mae: 0.0930 - root_mean_squared_error: 0.1366
Epoch 180/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0187 - mse: 0.0187 - mae: 0.0930 - root_mean_squared_error: 0.1366
Epoch 181/200
1/1 [=

In [24]:
factors = model_prediction.weights[0].numpy().reshape(n_forecast)
for t in range(n_forecast):
    print(f'factor t+{t+1} = {factors[t]:.4f}')

factor t+1 = -0.0408
factor t+2 = -0.0125
factor t+3 = 0.0185
factor t+4 = -0.0063
factor t+5 = -0.0030
factor t+6 = -0.0240
factor t+7 = 0.0158
factor t+8 = 0.0023
factor t+9 = -0.0264
factor t+10 = -0.0402
factor t+11 = 0.0159
factor t+12 = -0.0229
factor t+13 = 0.0010
factor t+14 = 0.0068
factor t+15 = 0.1051
factor t+16 = 0.1561
factor t+17 = 0.1105
factor t+18 = 0.0947
factor t+19 = 0.0675
factor t+20 = 0.1453


In [25]:
error_1 = compute_error(Y_test_pred_real_1, Y_test_real)
error_1['model'] = 'model 1'
error_2 = compute_error(Y_test_pred_real_2, Y_test_real_2)
error_2['model'] = 'model trends'
error_assembled = compute_error(Y_test_assembled_pred_real, Y_test_assembled_real)
error_assembled['model'] = 'assembled'
pd.concat([error_1, error_2, error_assembled]).set_index('model')

,MAE(t+1),MSE(t+1),MAE(t+2),MSE(t+2),MAE(t+3),MSE(t+3),MAE(t+4),MSE(t+4),MAE(t+5),MSE(t+5),...,MAE(t+16),MSE(t+16),MAE(t+17),MSE(t+17),MAE(t+18),MSE(t+18),MAE(t+19),MSE(t+19),MAE(t+20),MSE(t+20)
model,,,,,,,,,,,,,,,,,,,,,
model 1,74.246497,10661.181702,64.859612,8287.868036,54.875115,5.992706e+03,48.304868,5066.573934,69.233385,9537.989886,...,291.965565,141194.272971,147.297524,40577.414599,211.333142,72714.931442,237.669351,92290.728277,279.445759,128032.712031
model trends,615.283346,619318.162119,597.649313,606784.083552,708.906178,1.034760e+06,600.412266,601334.015551,641.838813,755432.050846,...,494.508663,435262.153752,494.572441,438693.819976,475.924753,404937.398773,489.817600,433661.971431,480.883558,415089.513832
assembled,54.494289,6331.683331,60.420886,7426.896690,62.566905,7.438153e+03,46.831250,4913.244302,68.233721,9358.484411,...,320.721673,170132.963148,171.359895,50217.614753,231.425384,86758.686110,251.248726,103355.435355,305.335133,153550.177272
